<a href="https://colab.research.google.com/github/zyfer416/Construction-analysis/blob/main/code_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-magic opencv-python tensorflow keras numpy matplotlib

In [ ]:
# ------------------- 📂 UPLOAD AN IMAGE -------------------
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print(f"📂 Uploaded file: {image_path}")

Saving imggff.jpg to imggff.jpg
📂 Uploaded file: imggff.jpg


In [ ]:
# ------------------- 🔍 IMAGE VALIDATION & CONSTRUCTION DETECTION -------------------
import magic
import logging
import os
import numpy as np
from datetime import datetime
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

# ✅ Logging setup
def setup_logger():
    logger = logging.getLogger("ImageValidator")
    logger.setLevel(logging.DEBUG)
    handler = logging.FileHandler("image_validation.log")
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    handler.setFormatter(formatter)
    if not logger.hasHandlers():
        logger.addHandler(handler)
    logger.info("Logger initialized.")
    return logger

logger = setup_logger()

# ✅ Check file type
def initialize_magic():
    try:
        mime = magic.Magic(mime=True)
        logger.info("python-magic initialized.")
        return mime
    except Exception as e:
        logger.error(f"Magic init error: {e}")
        raise

def is_valid_image(image_path):
    try:
        mime = initialize_magic()
        file_type = mime.from_file(image_path)
        logger.info(f"File type: {file_type}")
        return file_type.startswith("image/")
    except Exception as e:
        logger.error(f"Validation error: {e}")
        return False

# 🏗️ Detect construction site using VGG16 ImageNet
def is_construction_site(image_path):
    try:
        model = VGG16(weights="imagenet")
        image = load_img(image_path, target_size=(224, 224))
        image_array = preprocess_input(np.expand_dims(img_to_array(image), axis=0))
        predictions = model.predict(image_array)
        decoded_predictions = decode_predictions(predictions, top=50)[0]

        print("\n🔹 Top Predictions (ImageNet):")
        for i, (_, label, score) in enumerate(decoded_predictions[:10]):
            print(f"{i+1}. {label}: {score*100:.2f}%")

        construction_keywords = [
            "crane", "scaffold", "construction site", "bulldozer", "excavator", "altar", "building",
    "foundation work", "concrete pouring", "rebar reinforcement", "steel framework", "bricklaying",
    "masonry work", "construction workers", "heavy machinery", "road construction", "high-rise construction",
    "plastering work", "painting construction", "roof construction", "structural engineering",
    "construction site inspection", "tower crane", "earthmoving equipment", "site excavation",
    "bridge construction", "tunnel construction", "construction materials", "formwork setup",
    "site development", "pavement construction", "cement mixing","steel_arch_bridge", "construction debris"
        ]

        for _, label, _ in decoded_predictions:
            for keyword in construction_keywords:
                if keyword in label.lower():
                    logger.info(f"Construction-related keyword found: {label}")
                    return True
        logger.info("No construction-related keywords found.")
        return False

    except Exception as e:
        logger.error(f"Construction detection error: {e}")
        return False


INFO:ImageValidator:Logger initialized.


In [ ]:
# --- STEP 1: TRAINING SETUP ---
from google.colab import drive
drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset paths
data_dir = '/content/drive/MyDrive/CONSTRUCTION DATASET/NewSIH_IMAGES'

# Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

# Load data
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True)
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Load and modify VGG16
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
for param in model.features.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(model.classifier[6].in_features, len(class_names))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)

# Train model
num_epochs = 10
for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}\n' + '-'*10)
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("\n✅ Training Complete!")

# Save model
model_path = "/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved at: {model_path}")


In [ ]:
# --- STEP 2: PREDICT AFTER TRAINING ---
import torch
from torchvision import transforms
from PIL import Image
import torchvision.models as models
import torch.nn as nn

# Class names
class_names = ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']

# Load model
model_path = "/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth"
model = models.vgg16(weights=None)
model.classifier[6] = nn.Linear(4096, len(class_names))
model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
model.eval()

# Transform input
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Predict function
def predict_stage(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
    scores = torch.nn.functional.softmax(outputs[0], dim=0)

    print("\n🔍 Stage Prediction Scores:")
    for idx, score in enumerate(scores):
        print(f"{class_names[idx]}: {score.item()*100:.2f}%")

    predicted_stage = class_names[scores.argmax()]
    print(f"\n✅ Final Construction Stage Prediction: {predicted_stage}")


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.0/973.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from ultralytics import YOLO
import cv2
from IPython.display import Image as IPyImage, display

# Load a pretrained YOLOv8 model (YOLOv8n is fastest, you can choose yolov8s/yolov8m/yolov8l too)
yolo_model = YOLO("yolov8n.pt")


In [ ]:
# ------------------- 🚀 FULL PIPELINE EXECUTION -------------------
if is_valid_image(image_path):
    print("✅ Step 1: Image validation passed.")

    if is_construction_site(image_path):
        print("✅ Step 2: Construction site detected.")

        print("🔁 Step 3: Predicting construction stage...")
        predict_stage(image_path)
    else:
        print("⚠️ Step 2: Image does not appear to be a construction site.")
else:
    print("❌ Step 1: Not a valid image file. Please upload a valid image.")


INFO:ImageValidator:python-magic initialized.
INFO:ImageValidator:File type: image/jpeg


✅ Step 1: Image validation passed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step


INFO:ImageValidator:Construction-related keyword found: crane



🔹 Top Predictions (ImageNet):
1. dam: 18.23%
2. dome: 15.25%
3. triumphal_arch: 9.73%
4. mosque: 5.04%
5. pier: 4.22%
6. planetarium: 4.09%
7. crane: 3.67%
8. forklift: 2.18%
9. pedestal: 2.11%
10. freight_car: 1.83%
✅ Step 2: Construction site detected.
🔁 Step 3: Predicting construction stage...

🔍 Stage Prediction Scores:
Brickwork: 0.03%
Foundation: 0.02%
Framework: 96.32%
Painting: 0.03%
Plastering: 3.59%

✅ Final Construction Stage Prediction: Framework
